In [34]:
import pandas as pd
import os
from dateutil.parser import parse

In [35]:
import pandas as pd
import os
from dateutil.parser import parse

def convert_volume(vol):
    if pd.isna(vol) or vol in ['-', '']:
        return None  # Return None for NaN values or empty strings
    try:
        if 'K' in vol:
            return float(vol.replace('K', '')) * 1e3
        elif 'M' in vol:
            return float(vol.replace('M', '')) * 1e6
        else:
            return float(vol)
    except ValueError:
        return None  # Return None if the conversion fails

def try_parse_date(date_str, is_uk_format):
    try:
        if is_uk_format:
            return parse(date_str, dayfirst=True)
        else:
            # Convert US format to UK format
            return parse(date_str, dayfirst=False)
    except ValueError:
        print(f"Unable to parse date: {date_str}")
        return pd.NaT  # Return NaT for unparseable dates

directory = 'Investing.com'
combined_data = pd.DataFrame()

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        data = pd.read_csv(file_path)

        # Check if any date in the 'Date' column has the first two digits greater than 12
        is_uk_format = any(int(date.split('/')[0]) > 12 for date in data['Date'])
        
        # Apply the volume conversion function
        data['Vol.'] = data['Vol.'].apply(convert_volume)

        # Parse dates using custom date parser
        data['Date'] = data['Date'].apply(lambda x: try_parse_date(x, is_uk_format))

        # Drop rows with NaT (unparseable dates)
        data = data.dropna(subset=['Date'])

        # Set 'Date' as the index
        data.set_index('Date', inplace=True)

        # Rename the 'Price' and 'Vol.' columns to include the file name as a prefix
        data.rename(columns={'Price': f'{filename[:-4]}_price', 'Vol.': f'{filename[:-4]}_volume'}, inplace=True)

        # Select only the renamed price and volume columns
        data = data[[f'{filename[:-4]}_price', f'{filename[:-4]}_volume']]

        # Concatenate this data to the combined DataFrame
        if combined_data.empty:
            combined_data = data
        else:
            combined_data = combined_data.join(data, how='outer')

# Sort the DataFrame by the index (Date) after all data has been joined
combined_data.sort_index(inplace=True)

# Save the combined DataFrame to a pickle file
combined_data.to_pickle('raw_features.pkl')


In [36]:
combined_data.tail()

IEAC Historical Data_price  IEAC Historical Data_volume  \
Date                                                                  
2024-11-04                      118.11                     724040.0   
2024-12-01                      117.79                    1460000.0   
2024-12-02                      117.36                      21880.0   
2024-12-03                      118.20                     350710.0   
2024-12-04                      118.58                     417960.0   

            TCRS ETF Stock Price History_price  \
Date                                             
2024-11-04                                 NaN   
2024-12-01                                 NaN   
2024-12-02                                 NaN   
2024-12-03                                 NaN   
2024-12-04                                 NaN   

            TCRS ETF Stock Price History_volume  XTC5 Historical Data_price  \
Date                                                                          
2024-11-04                                  NaN                       31.74   
2024-12-01                                  NaN                       31.68   
2024-12-02                                  NaN                       31.70   
2024-12-03                                  NaN                       31.24   
2024-12-04                                  NaN                       31.90   

            XTC5 Historical Data_volume  
Date                                     
2024-11-04                          NaN  
2024-12-01                       2270.0  
2024-12-02                        350.0  
2024-12-03                          NaN  
2024-12-04                          NaN